In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
from _InternalFunctions.calcular_puntuacion_por_fase import calcular_puntuacion_por_fase
from _InternalFunctions.clean_outliers import clean_outliers
from _InternalFunctions.correlacion_numericas_target import correlacion_numericas_target
from _InternalFunctions.create_new_columns import create_new_columns
from _InternalFunctions.delete_columns import delete_columns
from _InternalFunctions.delete_records import delete_records
from _InternalFunctions.format_date import format_date
from _InternalFunctions.format_numeric import format_numeric
from _InternalFunctions.graficar_correlaciones import graficar_correlaciones
from _InternalFunctions.load_data import load_data
from _InternalFunctions.plot_boxplot import plot_boxplot
from _InternalFunctions.plot_histogram_subplots import plot_histogram_subplots
from _InternalFunctions.plot_sobrecosto_distribution import plot_sobrecosto_distribution
from _InternalFunctions.realizar_anova_con_graficos import realizar_anova_con_graficos
from _InternalFunctions.rename_columns import rename_columns

#### Clean Process

In [ ]:
project=load_data('Input_output/Data_input/Capital_Project_Schedules_and_Budgets.csv')

In [ ]:
project.info()

In [ ]:
# Renombrar columnas
rename_dict={
    'Project Geographic District ':'Distrito Geografico',
    'Project Building Identifier':'ID Edificio',
    'Project School Name':'Nombre Escuela',
    'Project Type ':'Tipo de Proyecto',
    'Project Description':'Descripción del Proyecto',
    'Project Phase Name':'Fase del Proyecto',
    'Project Status Name':'Estado del Proyecto',
    'Project Phase Actual Start Date':'Fecha de Inicio',
    'Project Phase Planned End Date':'Fecha de Fin Proyectada',
    'Project Phase Actual End Date':'Fecha de Fin Real',
    'Project Budget Amount':'Presupuesto del Proyecto',
    'Final Estimate of Actual Costs Through End of Phase Amount':'Presupuesto Final Estimado',
    'Total Phase Actual Spending Amount':'Cantidad Real Gastada',
    'DSF Number(s)':'Identificador DSF'
}
project=rename_columns(project, rename_dict)

In [ ]:
# Eliminar registros con valores incoherentes
filtro1=project['Presupuesto del Proyecto'].isin(['DOER', '0', 'DIIR', 'DOES', 'IEH', 'FTK', 'EMER', 'DIIT', 'DOEL', 'TPL', 'DOEP'])
filtro2=project['Estado del Proyecto'].isin(['PNS'])

project=delete_records(project, filtro1)
project=delete_records(project, filtro2)


In [ ]:
# Cambiar formato de variables
project=format_date(project, 'Fecha de Inicio')
project=format_date(project, 'Fecha de Fin Proyectada')
project=format_date(project, 'Fecha de Fin Real')
project=format_numeric(project, 'Presupuesto del Proyecto')

In [ ]:
# Eliminar columnas innecesarias
project=delete_columns(project, columns=['Identificador DSF', 'ID Edificio'])

In [ ]:
# Creación de nuevas columnas a partir de las existentes
project=create_new_columns(project,'Duración Proyectada',
                            (project['Fecha de Fin Proyectada']-project['Fecha de Inicio']).dt.days)

project=create_new_columns(project,'Duración Real',
                            (project['Fecha de Fin Real']-project['Fecha de Inicio']).dt.days)

project=create_new_columns(project,'Retraso',
                            (project['Fecha de Fin Real']-project['Fecha de Fin Proyectada']).dt.days)

project=create_new_columns(project,'Desviación Presupuestaria',
                           round(((project['Cantidad Real Gastada']-project['Presupuesto del Proyecto'])/project['Presupuesto del Proyecto'])*100,2))


In [ ]:
# Eliminar columnas innecesarias
project=delete_columns(project, columns=['Fecha de Fin Real', 'Fecha de Fin Proyectada', 'Fecha de Inicio'])

In [ ]:
# Eliminar proyectos con duración real cero o negativa y con cantidad real gastada cero o negativa
filtro3=project['Duración Real']<=0
filtro4=project['Cantidad Real Gastada']<=0
project=delete_records(project, filtro3)
project=delete_records(project, filtro4)

In [ ]:
# dividir la base de datos en dos partes: proyectos completos y en progreso
# Proyectos completos
project_complete=project[project['Estado del Proyecto']=='Complete']
project_complete.dropna(inplace=True)
# Proyectos en progreso
project_inprogress=project[project['Estado del Proyecto']=='In-Progress']
project_inprogress=delete_columns(project_inprogress, columns=['Duración Real','Retraso'])
project_inprogress.dropna(inplace=True)

In [ ]:
# Guardar los DataFrames
project_complete.to_csv('Input_output/Data_output/Capital_Project_Schedules_and_Budgets_clean_complete.csv', index=False)
project_inprogress.to_csv('Input_output/Data_output/Capital_Project_Schedules_and_Budgets_clean_inprogress.csv', index=False)

#### Working with DataFrame_Complete

In [ ]:
project_complete=load_data('Input_output/Data_output/Capital_Project_Schedules_and_Budgets_clean_complete.csv')

In [ ]:
plot_sobrecosto_distribution(project_complete, 'Desviación Presupuestaria')

In [ ]:
project_complete.describe()

In [ ]:
# histograma de presupuestos y gastos
plot_histogram_subplots(project_complete,
                       ['Presupuesto del Proyecto','Presupuesto Final Estimado','Cantidad Real Gastada'], 
                       title='Distribución de Presupuestos y Gastos', 
                       bins=30, 
                       log_scale=True)

In [ ]:
# boxplot de duraciones y retrasos
plot_boxplot(project_complete, 
             ['Duración Proyectada', 'Duración Real', 'Retraso'], 
             title='Distribución de Duraciones y Retrasos')

In [ ]:
project_clean_complete=clean_outliers(project_complete, columns=['Presupuesto del Proyecto', 'Presupuesto Final Estimado', 'Cantidad Real Gastada', 
                                                                'Duración Proyectada', 'Duración Real', 'Retraso','Desviación Presupuestaria'])
project_clean_complete=project_clean_complete[~project_clean_complete['Presupuesto Final Estimado'].isin([28,186,176,179,164,154])]
project_clean_complete=project_clean_complete[~project_clean_complete['Duración Real'].isin(range(1,30))]
project_clean_complete=project_clean_complete[~project_clean_complete['Cantidad Real Gastada'].isin([6,15,59,74])]
project_clean_complete=delete_records(project_clean_complete, project_clean_complete['Fase del Proyecto']=='Construction')

In [ ]:
project_clean_complete.describe()

In [ ]:
plot_histogram_subplots(project_clean_complete,
                       ['Presupuesto del Proyecto','Presupuesto Final Estimado','Cantidad Real Gastada'], 
                       title='Distribución de Presupuestos y Gastos', 
                       bins=30, 
                       log_scale=True)

In [ ]:
# boxplot de duraciones y retrasos
plot_boxplot(project_clean_complete, 
             ['Duración Proyectada', 'Duración Real', 'Retraso'], 
             title='Distribución de Duraciones y Retrasos')

In [ ]:
sns.histplot(data=project_clean_complete, x='Desviación Presupuestaria', kde=True, bins=30)
plt.title('Distribución de Desviación Presupuestaria')
plt.xlabel('Porcentaje de Desviación')
plt.ylabel('Frecuencia')
plt.gca().spines["top"].set_visible(False)  # Ocultar borde superior
plt.gca().spines["right"].set_visible(False)  # Ocultar borde derecho
plt.show()

In [ ]:
palette = {
    'Scope': 'blue',
    'Design': 'green',
    'CM,F&E': 'orange'
}
sns.catplot(x='Fase del Proyecto', y='Sobrecosto', data=project_clean_complete, kind='box', palette=palette, height=6, aspect=2)
plt.title('Desviación Presupuestaria por Fase del Proyecto')
plt.xlabel('Fase del Proyecto')
plt.ylabel('Sobrecosto (%)')
plt.show()

In [ ]:
#organizar columnas
column_order=['Distrito Geografico', 'Nombre Escuela', 'Tipo de Proyecto',
       'Descripción del Proyecto', 'Fase del Proyecto', 'Estado del Proyecto',
       'Presupuesto del Proyecto', 'Presupuesto Final Estimado',
       'Desviación Presupuestaria','Duración Proyectada', 'Duración Real',
       'Retraso', 'Cantidad Real Gastada']
project_clean_complete=project_clean_complete[column_order]

filtered_data = project_clean_complete.select_dtypes('number')

# Calcular la matriz de correlación
correlation_matrix = filtered_data.corr()

# Crear el heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap="coolwarm", fmt=".2f")
plt.title("Heatmap de Correlación")
plt.show()

In [ ]:
#crear la columna categoria proyecto a partir de la descripcion del proyecto

#definir las categorias y palabras clave
categoria_palabras = {
    "Mejoras exteriores": ['exterior', 'masonry', 'roofs', 'parapets', 'windows', 'replacement'],
    "Sistemas de seguridad": ['fire', 'alarm', 'camera', 'system', 'ipdvs', 'installations'],
    "Control climático": ['climate', 'ventilation', 'boiler', 'freezers', 'electrical', 'low', 'voltage'],
    "Infraestructura escolar": ['auditorium', 'playground', 'redevelopment', 'walk', 'accessibility'],
    "Proyectos ambientales": ['flood', 'elimination', 'planyc'],
    "Transformaciones internas": ['conversion', 'upgrade', 'installation', 'systems'],
    "COVID / Salud": ['covid', 'ida'],
    "Transporte / movilidad": ['path', 'travel'],
    "Programas educativos": ['program', 'full', 'reso']
}

def asignar_categoria(descripcion):
    descripcion_lower = str(descripcion).lower()
    for categoria, palabras in categoria_palabras.items():
        if any(palabra in descripcion_lower for palabra in palabras):
            return categoria
    return 'Otro'

project_clean_complete['Categoria Proyecto'] = project_clean_complete['Descripción del Proyecto'].apply(asignar_categoria)

# eliminar columna descripcion del proyecto
project_clean_complete=delete_columns(project_clean_complete, columns=['Descripción del Proyecto'])

In [ ]:
#Renombrar los tipos de proyecto para que sean más descriptivos
renombrar_tipo_proyecto = {
    "SCA CIP": "Proyecto de Mejoramiento General (CIP)",
    "SCA CIP RESOA": "Mejoras Asociadas a Programas Académicos (RESOA)",
    "3K": "Adecuaciones para Educación Inicial 3K",
    "SCA Lease Site Improvement": "Mejoras en Sedes Arrendadas",
    "SCA Capacity": "Expansión de Capacidad Escolar",
    "PRE-K": "Adecuaciones para Educación Preescolar (Pre-K)",
    "SCA Emergency Lighting": "Instalación de Iluminación de Emergencia"
}

# Crear nueva columna con nombres más descriptivos
project_clean_complete["Tipo Proyecto"] = project_clean_complete["Tipo de Proyecto"].map(renombrar_tipo_proyecto).fillna("Otro")
# eliminar columna tipo de proyecto
project_clean_complete=delete_columns(project_clean_complete, columns=['Tipo de Proyecto'])

#organizar columnas en el orden deseado
column_order=['Distrito Geografico', 'Nombre Escuela', 'Tipo Proyecto',
       'Categoria Proyecto', 'Fase del Proyecto', 'Estado del Proyecto',
       'Presupuesto del Proyecto', 'Presupuesto Final Estimado',
       'Desviación Presupuestaria','Duración Proyectada', 'Duración Real',
       'Retraso', 'Cantidad Real Gastada']
project_clean_complete=project_clean_complete[column_order]

In [ ]:
sns.pairplot(data=project_clean_complete, vars=['Presupuesto del Proyecto', 'Presupuesto Final Estimado', 'Cantidad Real Gastada',
                                                'Duración Proyectada', 'Duración Real', 'Retraso','Desviación Presupuestaria'])
plt.show()

In [ ]:
project_clean_complete.to_csv('Input_output/Data_output_transform/Project_complete.csv', index=False)

In [ ]:
# proyectos que gastaron más de lo presupuestado y terminaron antes de tiempo                       
project_clean_complete[(project_clean_complete['Cantidad Real Gastada']>project_clean_complete['Presupuesto del Proyecto'])&
                       (project_clean_complete['Retraso']<0)
                       &(project_clean_complete['Desviación Presupuestaria']>0)]

# proyectos que gastaron menos de lo presupuestado y terminaron después de tiempo
project_clean_complete[(project_clean_complete['Cantidad Real Gastada']<project_clean_complete['Presupuesto del Proyecto'])&
                       (project_clean_complete['Retraso']>0)
                       &(project_clean_complete['Desviación Presupuestaria']<0)]

#proyectos que gastaron más de lo presupuestado y terminaron después de tiempo
project_clean_complete[(project_clean_complete['Cantidad Real Gastada']>project_clean_complete['Presupuesto del Proyecto'])&
                       (project_clean_complete['Retraso']>0)
                       &(project_clean_complete['Desviación Presupuestaria']>0)]

#proyectos que gastaron menos de lo presupuestado y terminaron antes de tiempo
project_clean_complete[(project_clean_complete['Cantidad Real Gastada']<project_clean_complete['Presupuesto del Proyecto'])&
                        (project_clean_complete['Retraso']<0)
                        &(project_clean_complete['Desviación Presupuestaria']<0)]

In [ ]:
project_clean_complete.columns

#### working with DataFrame_In-Progress

In [ ]:
project_inprogress=load_data('Input_output/Data_output/Capital_Project_Schedules_and_Budgets_clean_inprogress.csv')

In [ ]:
project_clean_progress=clean_outliers(project_inprogress, columns=['Presupuesto del Proyecto', 'Presupuesto Final Estimado', 'Cantidad Real Gastada', 
                                                                'Duración Proyectada','Desviación Presupuestaria'])
project_clean_progress=project_clean_progress[~project_clean_progress['Duración Proyectada'].isin([34,59,89,90,91,92])]
project_clean_progress=project_clean_progress[~project_clean_progress['Cantidad Real Gastada'].isin([36,53,112,160,169])]
project_clean_progress=project_clean_progress[project_clean_progress['Desviación Presupuestaria']<-10]

In [ ]:
project_clean_progress.describe()

In [ ]:
plot_histogram_subplots(project_clean_progress,
                       ['Presupuesto del Proyecto','Presupuesto Final Estimado','Cantidad Real Gastada'], 
                       title='Distribución de Presupuestos y Gastos', 
                       bins=30, 
                       log_scale=True)

In [ ]:
filtered_data = project_clean_progress.select_dtypes('number')

# Calcular la matriz de correlación
correlation_matrix = filtered_data.corr()

# Crear el heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap="coolwarm", fmt=".2f")
plt.title("Heatmap de Correlación")
plt.show()

In [ ]:
#crear la columna categoria proyecto a partir de la descripcion del proyecto

#definir las categorias y palabras clave
categoria_palabras = {
    "Mejoras exteriores": ['exterior', 'masonry', 'roofs', 'parapets', 'windows', 'replacement'],
    "Sistemas de seguridad": ['fire', 'alarm', 'camera', 'system', 'ipdvs', 'installations'],
    "Control climático": ['climate', 'ventilation', 'boiler', 'freezers', 'electrical', 'low', 'voltage'],
    "Infraestructura escolar": ['auditorium', 'playground', 'redevelopment', 'walk', 'accessibility'],
    "Proyectos ambientales": ['flood', 'elimination', 'planyc'],
    "Transformaciones internas": ['conversion', 'upgrade', 'installation', 'systems'],
    "COVID / Salud": ['covid', 'ida'],
    "Transporte / movilidad": ['path', 'travel'],
    "Programas educativos": ['program', 'full', 'reso']
}

def asignar_categoria(descripcion):
    descripcion_lower = str(descripcion).lower()
    for categoria, palabras in categoria_palabras.items():
        if any(palabra in descripcion_lower for palabra in palabras):
            return categoria
    return 'Otro'

project_clean_progress['Categoria Proyecto'] = project_clean_progress['Descripción del Proyecto'].apply(asignar_categoria)

# eliminar columna descripcion del proyecto
project_clean_progress=delete_columns(project_clean_progress, columns=['Descripción del Proyecto'])

In [ ]:
#Renombrar los tipos de proyecto para que sean más descriptivos
renombrar_tipo_proyecto = {
    "SCA CIP": "Proyecto de Mejoramiento General (CIP)",
    "SCA CIP RESOA": "Mejoras Asociadas a Programas Académicos (RESOA)",
    "3K": "Adecuaciones para Educación Inicial 3K",
    "SCA Lease Site Improvement": "Mejoras en Sedes Arrendadas",
    "SCA Capacity": "Expansión de Capacidad Escolar",
    "PRE-K": "Adecuaciones para Educación Preescolar (Pre-K)",
    "SCA Emergency Lighting": "Instalación de Iluminación de Emergencia"
}

# Crear nueva columna con nombres más descriptivos
project_clean_progress["Tipo Proyecto"] = project_clean_progress["Tipo de Proyecto"].map(renombrar_tipo_proyecto).fillna("Otro")
# eliminar columna tipo de proyecto
project_clean_progress=delete_columns(project_clean_progress, columns=['Tipo de Proyecto'])

#organizar columnas en el orden deseado
column_order=['Distrito Geografico', 'Nombre Escuela', 'Tipo Proyecto',
       'Categoria Proyecto', 'Fase del Proyecto', 'Estado del Proyecto',
       'Presupuesto del Proyecto', 'Presupuesto Final Estimado',
       'Desviación Presupuestaria','Duración Proyectada',
       'Cantidad Real Gastada']
project_clean_progress=project_clean_progress[column_order]

project_clean_progress=rename_columns(project_clean_progress, {'Cantidad Real Gastada':'Cantidad Gastada a la Fecha'})

In [ ]:
project_clean_progress.to_csv('Input_output/Data_output_transform/Project_progress.csv', index=False)

In [ ]:
project_clean_progress.columns

#### Creating features and the working dataframe

In [ ]:
project=load_data('Input_output/Data_output_transform/Project_complete.csv')
project_2=load_data('Input_output/Data_output_transform/Project_progress.csv')
prob_imp=pd.read_csv('Input_output/Data_input/Probabilidad_Impacto.csv', sep=';')

In [ ]:
# Creación de dos nuevas variables: Probabilidad e Impacto

# Calular la puntuación de probabilidad e impacto por fase, retraso y presupuesto
result1 = calcular_puntuacion_por_fase(prob_imp, 'Fase', 'Media de Probabilidad', 'Impacto Medio', 'Scope')
result2 = calcular_puntuacion_por_fase(prob_imp, 'Fase', 'Media de Probabilidad', 'Impacto Medio', 'Design')
result3 = calcular_puntuacion_por_fase(prob_imp, 'retraso >30%', 'Media de Probabilidad', 'Impacto Medio', 'Retraso')
result4 = calcular_puntuacion_por_fase(prob_imp, 'rango 1', 'Media de Probabilidad', 'Impacto Medio', 'Presupuesto')
result5 = calcular_puntuacion_por_fase(prob_imp, 'rango 2', 'Media de Probabilidad', 'Impacto Medio', 'Presupuesto')
result6 = calcular_puntuacion_por_fase(prob_imp, 'rango 3', 'Media de Probabilidad', 'Impacto Medio', 'Presupuesto')
result7 = calcular_puntuacion_por_fase(prob_imp, 'rango 4', 'Media de Probabilidad', 'Impacto Medio', 'Presupuesto')

# Filtrar los valores donde 'Fase del Proyecto' sea 'Scope' y asignar result1
project.loc[project['Fase del Proyecto'] == 'Scope', 'ScopeP'] = result1[0]
project.loc[project['Fase del Proyecto'] == 'Scope', 'ScopeI'] = result1[1]

# Filtrar los valores donde 'Fase del Proyecto' sea 'Design' y asignar result2
project.loc[project['Fase del Proyecto'] == 'Design', 'DesignP'] = result2[0]
project.loc[project['Fase del Proyecto'] == 'Design', 'DesignI'] = result2[1]

# Filtrar los valores donde 'Retraso' sea 'retraso >30%' y asignar result3
project.loc[project['Retraso'] >= 0.3*project['Duración Proyectada'], 'RetrasoP'] = result3[0]
project.loc[project['Retraso'] >= 0.3*project['Duración Proyectada'], 'RetrasoI'] = result3[1]

# Filtrar los valores donde 'Fase del Proyecto' sea 'rango 1' y asignar result4
percentil_25 = project['Presupuesto del Proyecto'].quantile(0.25)
project.loc[project['Presupuesto del Proyecto'] <= percentil_25, 'rango1P'] = result4[0]
project.loc[project['Presupuesto del Proyecto'] <= percentil_25, 'rango1I'] = result4[1]

# Filtrar los valores donde 'Fase del Proyecto' sea 'rango 2' y asignar result5
percentil_75 = project['Presupuesto del Proyecto'].quantile(0.75)
project.loc[(project['Presupuesto del Proyecto'] > percentil_25) & (project['Presupuesto del Proyecto'] <= percentil_75), 'rango2P'] = result5[0]
project.loc[(project['Presupuesto del Proyecto'] > percentil_25) & (project['Presupuesto del Proyecto'] <= percentil_75), 'rango2I'] = result5[1]

# Filtrar los valores donde 'Fase del Proyecto' sea 'rango 3' y asignar result6
percentil_90 = project['Presupuesto del Proyecto'].quantile(0.90)
project.loc[(project['Presupuesto del Proyecto'] > percentil_75) & (project['Presupuesto del Proyecto'] <= percentil_90), 'rango3P'] = result6[0]
project.loc[(project['Presupuesto del Proyecto'] > percentil_75) & (project['Presupuesto del Proyecto'] <= percentil_90), 'rango3I'] = result6[1]

# Filtrar los valores donde 'Fase del Proyecto' sea 'rango 4' y asignar result7
project.loc[project['Presupuesto del Proyecto'] > percentil_90, 'rango4P'] = result7[0]
project.loc[project['Presupuesto del Proyecto'] > percentil_90, 'rango4I'] = result7[1]

#Creación de nuevas columnas a partir de las existentes
project['Probabilidad']=project[['ScopeP','DesignP','RetrasoP','rango1P','rango2P','rango3P','rango4P']].mean(axis=1)
project['Impacto']=project[['ScopeI','DesignI','RetrasoI','rango1I','rango2I','rango3I','rango4I']].mean(axis=1)

#Eliminar columnas innecesarias
project=delete_columns(project, columns=['ScopeP','DesignP','RetrasoP','rango1P','rango2P','rango3P','rango4P',
                                         'ScopeI','DesignI','RetrasoI','rango1I','rango2I','rango3I','rango4I'])

In [ ]:
# # Creación de modelo para estimar gasto a la fecha en proyectos completados

# from sklearn.ensemble import RandomForestRegressor
# from sklearn.preprocessing import OneHotEncoder
# from sklearn.compose import ColumnTransformer
# from sklearn.pipeline import Pipeline
# from sklearn.model_selection import train_test_split

# df_train = project_2.copy()

# # Entrenamiento del modelo con datos de proyectos en progreso
# X = df_train[['Tipo Proyecto',
#        'Categoria Proyecto', 'Fase del Proyecto',
#        'Presupuesto del Proyecto', 'Presupuesto Final Estimado',
#        'Desviación Presupuestaria', 'Duración Proyectada']]
# y = df_train['Cantidad Gastada a la Fecha']

# # División de datos para prueba/entrenamiento
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# # Codificación de variables categóricas
# categorical_cols = ['Tipo Proyecto', 'Categoria Proyecto', 'Fase del Proyecto']
# numeric_cols = ['Presupuesto del Proyecto', 'Presupuesto Final Estimado',
#                 'Desviación Presupuestaria', 'Duración Proyectada']

# preprocessor = ColumnTransformer([
#     ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_cols)
# ], remainder='passthrough')

# # Pipeline de entrenamiento
# modelo_gasto_a_fecha = Pipeline(steps=[
#     ('preprocessor', preprocessor),
#     ('regressor', RandomForestRegressor(n_estimators=100, random_state=42))
# ])

In [ ]:
# from sklearn.metrics import mean_absolute_error, r2_score
# from sklearn.model_selection import train_test_split

# # entrenar modelo en datos de entrenamiento
# modelo_gasto_a_fecha.fit(X_train, y_train)

# # Predecir en test
# y_pred = modelo_gasto_a_fecha.predict(X_test)

# # Calcular métricas
# mae = mean_absolute_error(y_test, y_pred)
# r2 = r2_score(y_test, y_pred)

# print(f"MAE:  ${mae:,.0f}")
# print(f"R²:   {r2:.2f}")

In [ ]:
# plt.scatter(y_test, y_pred, alpha=0.7)
# plt.xlabel("Gasto simulado real")
# plt.ylabel("Gasto simulado predicho")
# plt.title("Comparación real vs predicho")
# plt.plot([y.min(), y.max()], [y.min(), y.max()], 'r--')
# plt.grid(True)
# plt.show()

In [ ]:
# #Aplicar el modelo para generar la columna "gasto_a_la_fecha_estimado" en proyectos completados
# X_pred = project[['Tipo Proyecto',
#        'Categoria Proyecto', 'Fase del Proyecto',
#        'Presupuesto del Proyecto', 'Presupuesto Final Estimado',
#        'Desviación Presupuestaria', 'Duración Proyectada']]
# project['Gasto a la Fecha Estimado'] = modelo_gasto_a_fecha.predict(X_pred)

# #organizar columnas en el orden deseado
# column_order=['Tipo Proyecto',
#        'Categoria Proyecto', 'Fase del Proyecto',
#        'Presupuesto del Proyecto', 'Presupuesto Final Estimado',
#        'Desviación Presupuestaria','Duración Proyectada',
#        'Gasto a la Fecha Estimado','Probabilidad','Impacto',
#        'Cantidad Real Gastada']
# project=project[column_order]

In [ ]:
# from sklearn.compose import ColumnTransformer
# from sklearn.preprocessing import OneHotEncoder, StandardScaler

# steps=[('preprocessor', ColumnTransformer(
#   transformers=[
#       ('num', StandardScaler(), ['impacto','Presupuesto del Proyecto','probabilidad',
#                        'Duración Proyectada','Presupuesto Final Estimado','Cantidad Real Gastada']),
#       ('cat', OneHotEncoder(), ['Fase del Proyecto','Estado del Proyecto','Tipo de Proyecto'])
#   ]  
# ))
# ]


In [ ]:
# output_dir = os.path.join("Input_output", "Data")
# if not os.path.exists(output_dir):
#     os.makedirs(output_dir)

# project.to_csv(os.path.join(output_dir, "DataFrame.csv"), index=False)

In [ ]:
data=load_data('Input_output/Data/DataFrame.csv')

In [ ]:
filtered_data = data[['Presupuesto del Proyecto', 'Presupuesto Final Estimado','Desviación Presupuestaria',
                 'Duración Proyectada','Probabilidad','Impacto','Gasto a la Fecha Estimado',
                 'Cantidad Real Gastada']]

# Calcular la matriz de correlación
correlation_matrix = filtered_data.corr()

# Crear el heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap="coolwarm", fmt=".2f")
plt.title("Heatmap de Correlación")
plt.show()

In [ ]:
sns.pairplot(data=data, vars=['Presupuesto del Proyecto', 'Presupuesto Final Estimado','Desviación Presupuestaria',
                  'Duración Proyectada','Probabilidad','Impacto','Gasto a la Fecha Estimado',
                  'Cantidad Real Gastada'])
plt.show()

In [ ]:
correlaciones=correlacion_numericas_target(data, variables_numericas=['Gasto a la Fecha Estimado', 'Presupuesto Final Estimado',
                                                                      'Presupuesto del Proyecto','Probabilidad','Impacto','Duración Proyectada',
                                                                      'Desviación Presupuestaria']
                                                                    , target='Cantidad Real Gastada')
graficar_correlaciones(correlaciones)

In [ ]:
realizar_anova_con_graficos(data, variables_categoricas=['Tipo Proyecto','Categoria Proyecto','Fase del Proyecto'], 
                            target='Cantidad Real Gastada')

#### Modelling


In [ ]:
from _ModelFunctions.create_model_directory import create_model_directory
from _ModelFunctions.create_pipeline import create_pipeline
from _ModelFunctions.divide_project_data import divide_project_data
from _ModelFunctions.metrics_calculate_from_file import metrics_calculate_from_file
from _ModelFunctions.perform_grid_search_cv import perform_grid_search_cv
from _ModelFunctions.plot_feature_importance_from_file import plot_feature_importance_from_file
from _ModelFunctions.plot_model_predictions import plot_model_predictions

In [ ]:
# lectura de la data
project_2025=pd.read_csv('Input_output/Data/DataFrame.csv')

# división de la data en entrenamiento y prueba
x_train, x_test, y_train, y_test = divide_project_data(project_2025,
                                                       target='Cantidad Real Gastada',
                                                       test_size=0.2,
                                                       random_state=42)

#### LinearRegression

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import f_regression

#preproceamiento de la data
pipeline_LR=create_pipeline(LinearRegression(), model_name='LinearRegression')

# hiperparámetros para la búsqueda de cuadrícula
param_grid_LR={
    'feature_selection__k': [5, 10, 15, 17],
    'feature_selection__score_func': [f_regression], 
    'regressor__fit_intercept': [True, False]
}

#búsqueda de cuadrícula para encontrar los mejores hiperparámetros
model_LR=perform_grid_search_cv(pipeline_LR, 
                       param_grid_LR, 
                       cv=10, 
                       scoring='neg_mean_absolute_error')

#ajustar el modelo con los mejores hiperparámetros
model_LR.fit(x_train, y_train)
print("Best score: ", model_LR.best_score_)
print("Best parameters: ", model_LR.best_params_)

#guardar el modelo entrenado
create_model_directory(model_LR, model_name='LinearRegression.pkl.gz')

#cargar el modelo guardado
model_path_LR='files/models/LinearRegression.pkl.gz'

#caculo de metricas del modelo
metrics_calculate_from_file(model_path_LR, x_train, x_test, y_train, y_test,
                            output_path='files/metrics/metrics1.json')

#Grafico de ajuste del modelo
plot_model_predictions(model_path_LR, x_test, y_test)


#### Random Forest Regressor


In [ ]:
from sklearn.ensemble import RandomForestRegressor

#preproceamiento de la data
pipeline_RF=create_pipeline(RandomForestRegressor(), model_name='RandomForestRegressor')

# hiperparámetros para la búsqueda de cuadrícula
param_grid_RF={
    'regressor__n_estimators': [100, 200, 300],  
    'regressor__max_depth': [10, 20, 30, None],  
    'regressor__min_samples_split': [2, 5, 10],  
    'regressor__min_samples_leaf': [1, 2, 4],  
    'regressor__max_features': ['auto', 'sqrt', 'log2'],  
    'regressor__bootstrap': [True, False]  
}

#búsqueda de cuadrícula para encontrar los mejores hiperparámetros
model_RF=perform_grid_search_cv(pipeline_RF, 
                       param_grid_RF, 
                       cv=10, 
                       scoring='neg_mean_absolute_error')

#ajustar el modelo con los mejores hiperparámetros
model_RF.fit(x_train, y_train)
print("Best score: ", model_RF.best_score_)
print("Best parameters: ", model_RF.best_params_)

#guardar el modelo entrenado
create_model_directory(model_RF, model_name='RandomForestRegressor.pkl.gz')

#cargar el modelo guardado
model_path_RF='files/models/RandomForestRegressor.pkl.gz'

#caculo de metricas del modelo
metrics_calculate_from_file(model_path_RF, x_train, x_test, y_train, y_test,
                            output_path='files/metrics/metrics2.json')

# #Grafico de ajuste del modelo
# plot_model_predictions(model_path_RF, x_test, y_test)

#### Gradient Boosting Regressor


In [ ]:
from sklearn.ensemble import GradientBoostingRegressor

#preproceamiento de la data
pipeline_GB=create_pipeline(GradientBoostingRegressor(), model_name='GradientBoostingRegressor')

# hiperparámetros para la búsqueda de cuadrícula
param_grid_GB={
    'regressor__n_estimators': [100, 200, 300],  
    'regressor__learning_rate': [0.01, 0.05, 0.1], 
    'regressor__max_depth': [3, 5, 7], 
    'regressor__min_samples_split': [2, 5, 10],  
    'regressor__min_samples_leaf': [1, 2, 4],  
    'regressor__subsample': [0.8, 1.0],  
    'regressor__max_features': ['auto', 'sqrt', 'log2']  
}


#búsqueda de cuadrícula para encontrar los mejores hiperparámetros
model_GB=perform_grid_search_cv(pipeline_GB, 
                       param_grid_GB, 
                       cv=6, 
                       scoring='neg_mean_absolute_error')

#ajustar el modelo con los mejores hiperparámetros
model_GB.fit(x_train, y_train)
print("Best score: ", model_GB.best_score_)
print("Best parameters: ", model_GB.best_params_)

#guardar el modelo entrenado
create_model_directory(model_GB, model_name='GradientBoostingRegressor.pkl.gz')

#cargar el modelo guardado
model_path_GB='files/models/GradientBoostingRegressor.pkl.gz'

#caculo de metricas del modelo
metrics_calculate_from_file(model_path_GB, x_train, x_test, y_train, y_test,
                            output_path='files/metrics/metrics3.json')

#Grafico de ajuste del modelo
plot_model_predictions(model_path_GB, x_test, y_test)

#### MLP Regressor


In [ ]:
from sklearn.neural_network import MLPRegressor

#preproceamiento de la data
pipeline_MLP=create_pipeline(MLPRegressor(), model_name='MLPRegressor')

# hiperparámetros para la búsqueda de cuadrícula
param_grid_MLP={
    'regressor__hidden_layer_sizes': [(100,), (50, 50)],
    'regressor__activation': ['relu', 'tanh'],
    'regressor__solver': ['adam', 'sgd'],
    'regressor__alpha': [0.0001, 0.001, 0.01],
    'regressor__learning_rate': ['constant', 'adaptive']
}


#búsqueda de cuadrícula para encontrar los mejores hiperparámetros
model_MLP=perform_grid_search_cv(pipeline_MLP, 
                       param_grid_MLP, 
                       cv=6, 
                       scoring='neg_mean_absolute_error')

#ajustar el modelo con los mejores hiperparámetros
model_MLP.fit(x_train, y_train)
print("Best score: ", model_MLP.best_score_)
print("Best parameters: ", model_MLP.best_params_)

#guardar el modelo entrenado
create_model_directory(model_MLP, model_name='MLPRegressor.pkl.gz')

#cargar el modelo guardado
model_path_MLP='files/models/MLPRegressor.pkl.gz'

#caculo de metricas del modelo
metrics_calculate_from_file(model_path_MLP, x_train, x_test, y_train, y_test,
                            output_path='files/metrics/metrics4.json')

#Grafico de ajuste del modelo
plot_model_predictions(model_path_MLP, x_test, y_test)